In [1]:
import pandas as pd
import numpy as np

import xgboost
import lightgbm

import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as st
import time
import timeit

In [2]:
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.metrics import mean_squared_error

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [3]:
import sklearn.datasets as skds

mnist = skds.load_digits()
faces = skds.fetch_olivetti_faces()

In [4]:
x_mnist, y_mnist = mnist.data, mnist.target
x_faces, y_faces = faces.data, faces.target

In [5]:
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'XGBoost': XGBClassifier(),
    'AdaBoost': AdaBoostClassifier()
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
def stratified_cross_val_scores(clf, X, y, cv):
    accuracy_scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(clf, X, y, cv=cv, scoring='precision_macro')
    recall_scores = cross_val_score(clf, X, y, cv=cv, scoring='recall_macro')
    return accuracy_scores, precision_scores, recall_scores

In [45]:
def experiment(dataset_name, X, y, classifiers, cv):
    results = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    for clf_name, clf in classifiers.items():
        start_time = time.time()
        if clf_name in ['Random Forest', 'XGBoost', 'AdaBoost']:
            # Ensemble algorithms, test both one-vs-one and one-vs-rest
            clf_ovo = OneVsOneClassifier(clf)
            clf_ovr = OneVsRestClassifier(clf)
            accuracy_scores_ovo, precision_scores_ovo, recall_scores_ovo = stratified_cross_val_scores(clf_ovo, X_train, y_train, cv)
            accuracy_scores_ovr, precision_scores_ovr, recall_scores_ovr = stratified_cross_val_scores(clf_ovr, X_train, y_train, cv)
            runtime = time.time() - start_time
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name + ' (One-vs-One)',
                'Accuracy Mean': np.mean(accuracy_scores_ovo),
                'Accuracy CI': np.std(accuracy_scores_ovo) * 2,
                'Precision Mean': np.mean(precision_scores_ovo),
                'Precision CI': np.std(precision_scores_ovo) * 2,
                'Recall Mean': np.mean(recall_scores_ovo),
                'Recall CI': np.std(recall_scores_ovo) * 2,
                'Runtime': runtime
            })
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name + ' (One-vs-Rest)',
                'Accuracy Mean': np.mean(accuracy_scores_ovr),
                'Accuracy CI': np.std(accuracy_scores_ovr) * 2,
                'Precision Mean': np.mean(precision_scores_ovr),
                'Precision CI': np.std(precision_scores_ovr) * 2,
                'Recall Mean': np.mean(recall_scores_ovr),
                'Recall CI': np.std(recall_scores_ovr) * 2,
                'Runtime': runtime
            })
        else:
            accuracy_scores, precision_scores, recall_scores = stratified_cross_val_scores(clf, X_train, y_train, cv)
            runtime = time.time() - start_time
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name,
                'Accuracy Mean': np.mean(accuracy_scores),
                'Accuracy CI': np.std(accuracy_scores) * 2,
                'Precision Mean': np.mean(precision_scores),
                'Precision CI': np.std(precision_scores) * 2,
                'Recall Mean': np.mean(recall_scores),
                'Recall CI': np.std(recall_scores) * 2,
                'Runtime': runtime
            })
    return results

results_mnist = experiment('MNIST', x_mnist, y_mnist, classifiers, cv)
results_faces = experiment('Olivetti Faces', x_faces, y_faces, classifiers, cv)

results_df_mnist = pd.DataFrame(results_mnist)
results_df_faces = pd.DataFrame(results_faces)

/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyboardInterrupt: 

In [ ]:
print("Results for MNIST Dataset:")
print(results_df_mnist)

print("\nResults for Olivetti Faces Dataset:")
print(results_df_faces)

In [7]:
def experiment(dataset_name, X, y, classifiers, cv):
    results = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    print(f"Evaluating models on {dataset_name} dataset:")
    for clf_name, clf in classifiers.items():
        print(f"  Evaluating {clf_name}...")
        start_time = time.time()
        if clf_name in ['Random Forest', 'XGBoost', 'AdaBoost']:
            # Ensemble algorithms, test both one-vs-one and one-vs-rest
            clf_ovo = OneVsOneClassifier(clf)
            clf_ovr = OneVsRestClassifier(clf)
            accuracy_scores_ovo, precision_scores_ovo, recall_scores_ovo = stratified_cross_val_scores(clf_ovo, X_train, y_train, cv)
            accuracy_scores_ovr, precision_scores_ovr, recall_scores_ovr = stratified_cross_val_scores(clf_ovr, X_train, y_train, cv)
            runtime = time.time() - start_time
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name + ' (One-vs-One)',
                'Accuracy Mean': np.mean(accuracy_scores_ovo),
                'Accuracy CI': np.std(accuracy_scores_ovo) * 2,
                'Precision Mean': np.mean(precision_scores_ovo),
                'Precision CI': np.std(precision_scores_ovo) * 2,
                'Recall Mean': np.mean(recall_scores_ovo),
                'Recall CI': np.std(recall_scores_ovo) * 2,
                'Runtime': runtime
            })
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name + ' (One-vs-Rest)',
                'Accuracy Mean': np.mean(accuracy_scores_ovr),
                'Accuracy CI': np.std(accuracy_scores_ovr) * 2,
                'Precision Mean': np.mean(precision_scores_ovr),
                'Precision CI': np.std(precision_scores_ovr) * 2,
                'Recall Mean': np.mean(recall_scores_ovr),
                'Recall CI': np.std(recall_scores_ovr) * 2,
                'Runtime': runtime
            })
        else:
            accuracy_scores, precision_scores, recall_scores = stratified_cross_val_scores(clf, X_train, y_train, cv)
            runtime = time.time() - start_time
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name,
                'Accuracy Mean': np.mean(accuracy_scores),
                'Accuracy CI': np.std(accuracy_scores) * 2,
                'Precision Mean': np.mean(precision_scores),
                'Precision CI': np.std(precision_scores) * 2,
                'Recall Mean': np.mean(recall_scores),
                'Recall CI': np.std(recall_scores) * 2,
                'Runtime': runtime
            })
    return results

results_mnist = experiment('MNIST', x_mnist, y_mnist, classifiers, cv)
results_faces = experiment('Olivetti Faces', x_faces, y_faces, classifiers, cv)

results_df_mnist = pd.DataFrame(results_mnist)
results_df_faces = pd.DataFrame(results_faces)

Evaluating models on MNIST dataset:
  Evaluating Logistic Regression...


/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

  Evaluating Random Forest...
  Evaluating SVM...
  Evaluating K-Nearest Neighbors...
  Evaluating Decision Tree...
  Evaluating XGBoost...
  Evaluating AdaBoost...
Evaluating models on Olivetti Faces dataset:
  Evaluating Logistic Regression...


/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

  Evaluating Random Forest...


...

/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_cl

  Evaluating XGBoost...


/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_cl

  Evaluating AdaBoost...


/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_cl

In [8]:
print("Results for MNIST Dataset:")
print(results_df_mnist)

print("\nResults for Olivetti Faces Dataset:")
print(results_df_faces)

Results for MNIST Dataset:
  Dataset                   Classifier  Accuracy Mean  Accuracy CI  \
0   MNIST          Logistic Regression       0.961735     0.018061   
1   MNIST   Random Forest (One-vs-One)       0.971475     0.013450   
2   MNIST  Random Forest (One-vs-Rest)       0.972859     0.008144   
3   MNIST                          SVM       0.988165     0.007126   
4   MNIST          K-Nearest Neighbors       0.987466     0.015021   
5   MNIST                Decision Tree       0.846922     0.030104   
6   MNIST         XGBoost (One-vs-One)       0.949211     0.028321   
7   MNIST        XGBoost (One-vs-Rest)       0.966594     0.022304   
8   MNIST        AdaBoost (One-vs-One)       0.956153     0.012990   
9   MNIST       AdaBoost (One-vs-Rest)       0.950595     0.025053   

   Precision Mean  Precision CI  Recall Mean  Recall CI    Runtime  
0        0.963323      0.016367     0.961730   0.018107   0.909493  
1        0.969189      0.015537     0.967921   0.016171  41.8961

### Results for MNIST Dataset

| Dataset | Classifier                     | Accuracy Mean | Accuracy CI | Precision Mean | Precision CI | Recall Mean | Recall CI | Runtime   |
|---------|--------------------------------|---------------|-------------|----------------|--------------|-------------|-----------|-----------|
| MNIST   | Logistic Regression            | 0.961735      | 0.018061    | 0.963323       | 0.016367     | 0.961730    | 0.018107  | 0.909493  |
| MNIST   | Random Forest (One-vs-One)     | 0.971475      | 0.013450    | 0.969189       | 0.015537     | 0.967921    | 0.016171  | 41.896149 |
| MNIST   | Random Forest (One-vs-Rest)    | 0.972859      | 0.008144    | 0.970824       | 0.018237     | 0.971420    | 0.019999  | 41.896149 |
| MNIST   | SVM                            | 0.988165      | 0.007126    | 0.988531       | 0.006777     | 0.988100    | 0.006897  | 0.490840  |
| MNIST   | K-Nearest Neighbors            | 0.987466      | 0.015021    | 0.987954       | 0.014304     | 0.987412    | 0.014939  | 0.323228  |
| MNIST   | Decision Tree                  | 0.846922      | 0.030104    | 0.855142       | 0.028204     | 0.841942    | 0.052435  | 0.144685  |
| MNIST   | XGBoost (One-vs-One)           | 0.949211      | 0.028321    | 0.951438       | 0.027108     | 0.949463    | 0.028208  | 8.228090  |
| MNIST   | XGBoost (One-vs-Rest)          | 0.966594      | 0.022304    | 0.967081       | 0.021606     | 0.966494    | 0.022083  | 8.228090  |
| MNIST   | AdaBoost (One-vs-One)          | 0.956153      | 0.012990    | 0.957531       | 0.011558     | 0.956167    | 0.012537  | 33.286500 |
| MNIST   | AdaBoost (One-vs-Rest)         | 0.950595      | 0.025053    | 0.951988       | 0.024341     | 0.950650    | 0.025224  | 33.286500 |


### Results for Olivetti Faces Dataset

| Dataset        | Classifier                     | Accuracy Mean | Accuracy CI | Precision Mean | Precision CI | Recall Mean | Recall CI | Runtime     |
|----------------|--------------------------------|---------------|-------------|----------------|--------------|-------------|-----------|-------------|
| Olivetti Faces | Logistic Regression            | 0.946875      | 0.046771    | 0.959167       | 0.047258     | 0.952500    | 0.045826  | 30.630620   |
| Olivetti Faces | Random Forest (One-vs-One)     | 0.906250      | 0.034233    | 0.912833       | 0.090962     | 0.917500    | 0.062450  | 527.340780  |
| Olivetti Faces | Random Forest (One-vs-Rest)    | 0.868750      | 0.042390    | 0.862500       | 0.113896     | 0.860000    | 0.090000  | 527.340780  |
| Olivetti Faces | SVM                            | 0.903125      | 0.066732    | 0.899583       | 0.082344     | 0.907500    | 0.060415  | 4.657888    |
| Olivetti Faces | K-Nearest Neighbors            | 0.768750      | 0.060596    | 0.764333       | 0.066789     | 0.787500    | 0.068920  | 0.240279    |
| Olivetti Faces | Decision Tree                  | 0.534375      | 0.134629    | 0.502833       | 0.110356     | 0.525000    | 0.142302  | 19.812741   |
| Olivetti Faces | XGBoost (One-vs-One)           | 0.484375      | 0.106434    | 0.430798       | 0.129923     | 0.490000    | 0.091378  | 560.455800  |
| Olivetti Faces | XGBoost (One-vs-Rest)          | 0.771875      | 0.096014    | 0.736250       | 0.123029     | 0.780000    | 0.117898  | 560.455800  |
| Olivetti Faces | AdaBoost (One-vs-One)          | 0.771875      | 0.156375    | 0.737750       | 0.074148     | 0.787500    | 0.144914  | 1178.738458 |
| Olivetti Faces | AdaBoost (One-vs-Rest)         | 0.778125      | 0.082443    | 0.760792       | 0.091489     | 0.767500    | 0.046368  | 1178.738458 |


**Best Model for MNIST:**

SVM has the highest accuracy, precision, and recall, with a very low runtime and narrow confidence intervals, implying that it is the most reliable model for the MNIST dataset.



**Best Model for Olivetti Faces:**

Logistic Regression has the highest accuracy, precision, and recall, with moderate runtime and confidence intervals, making it the most reliable model for the Olivetti Faces dataset.